# Customize your case: PRISM precipitation
We set up a special way to run_neon with PRISM precipitation data.

**This is an optional tutorial** it's a little bit more advanced, but it will help you think about how to modify the model configuration to run new sites or model experiments.  

The `run_neon` script we used in the [introductory tutorial 1b](1b_NEON_Simulation_Intro.ipynb) created and ran a base case as well as a `.transient` case.   Here we'll also be adding an experimental `.PRISM.transient` case that reads in an alternative precipitation dataset from PRISM. 

---

## In this tutorial

The tutorial has several components. Below you will find steps to: 
1. Set up and run a simulation

*Extra credit* 

2. Create another clone for an experiment you're hoping to do.

<div class="alert alert-block alert-warning">

<b>NOTICE:</b> If you're running this notebook through the NCAR JupyterHub login, you need to be on a Cheyenne login node (NOT Casper).  

</div>

***

#### Set up your environment
It is important in order to have all the tools and packages you need to run simulations.

<div class="alert alert-block alert-info">
   
<b>TIP:</b> <i>If you're running on Cheyenne</i>, you may need to uncomment the the following two lines of code. This will set up your conda environment correctly.

<b>This is not required if your running CESM-Lab in the cloud.</b>

</div>

In [ ]:
#module purge
#module load conda ncarenv

<h1> 1. Create a new case </h1>

## 1.1 Navigate to your source code

In [ ]:
cd ~/CTSM/tools/site_and_regional

## 1.2 Create a new case with `run_neon.py`


The following code will:
- create a new case
- make a few modifications
- submit the simulation

**REMEMBER** PRISM cases can only be run for NEON sites in the lower 48 states (HI, AK, and PR won't work)

In [ ]:
# Change the 4-character NEON site below.

export neon_site='KONZ'   # This should likely be a site you've already run.
export experiment='PRISM'

In [ ]:
# then run_neon
./run_neon.py --neon-sites $neon_site  \
   --output-root ~/scratch/NEON_cases \
   --experiment $experiment \
   --setup-only \
   --overwrite \
   --prism

Since this doesn't always render correctly on websites, here the code from above

```
./run_neon.py --neon-sites $neon_site  \
   --output-root ~/scratch/NEON_cases \
   --experiment $experiment \
   --setup-only \
   --overwrite \
   --prism
```
Two new flags are being used here:
- `--experiment` adds an additional string to our case name
- `--setup-only` creates the case without running it
- `--overwrite` will let you overwrite an existing case.  
- `--prism` configures the case to use PRISM precipitation data  

----
# 2. Introduction to controling case configuration

## 2.1 Setup and Build
First, we'll setup and build our case manually

NOTE: As before, this takes some time

In [ ]:
cd ~/scratch/NEON_cases/${neon_site}.${experiment}.transient
./case.setup
qcmd -- ./case.build

## 2.2 What's different with my case?
XML files are used to control how cases are configured 

Before digging in too we can start to see how our PRISM.transient case is different.

In [ ]:
diff env_run.xml ../${neon_site}.PRISM.transient/env_run.xml 

In [ ]:
# These are the datm.streams for the PRISM.transient case, we'll just look at the end of the file
cat CaseDocs/datm.streams.xml | tail -26


**What are the differences between these two cases** based on their `env_run.xml` and `datm.streams.xml` files?

Where does the KONZ.transient case get it's precipitation data from?

These .xml changes set up high-level control over how your simulation is run.  What are some of the specifics related to the land model?  We can see this by looking at our `lnd_in` file.

---

## 2.2 `lnd_in` 

**What are the differences between these two cases**, based on their `lnd_in` files?


In [ ]:
cat CaseDocs/lnd_in | grep finidat

This is not an initial conditions file from a PRISM simulation. We'll have to change this.

<style> 
table td, table th, table tr {text-align:left !important;}
</style>
<div class="alert alert-block alert-info">
<b>REMEMBER:</b>
    
- The <i>lnd_in</i> file provides a high level summary of all the name list chagnes and files that are being used by CLM. 
- It can be found in the CaseDocs directory, or in your run directory. 
- You cannot directly modifiy the <i>lnd_in file</i>, instead users can modify <i>user_nl_clm</i>.

</div>

**Initial conditions dataset:** `finidat`
  - These are initial conditions files that we created by spinning up the model.  
  - Spin up requires starting the model from bare ground conditions (we call it a *coldstart*).
  - Spin up takes a few hundred years of simulations so that ecosystem carbon and nitrogen pools acheive steady state conditions (e.g. average net ecosystem exchange equals zero).  
  - Since this takes a long time, we provide initial conditions for you to start from.
  - **This also means that if you change model parameterizations, input data, or anything else you ahve to spin up the model again!** 

---

The `lnd_in` files are controlled by `user_nl_clm`.  We have to modify our user_nl_clm to point to the right initial conditions data

## 2.3 Submit the case

In [ ]:
echo "finidat='/scratch/data/NEONv2/PRISM.restart/$neon_site.postad.clm2.r.0318-01-01-00000.nc'" >> user_nl_clm

#Then check it worked as intended
./preview_namelists

cat CaseDocs/lnd_in | grep finidat

In [ ]:
./case.submit

In [ ]:
# Check to see taht your case is running 
qstat -u $USER

In [ ]:
cat CaseStatus | tail -10

# 3. Create an experimental clone:

<h4> This step is optional, but provides helpful information that you may use in your own workflow </h4>

So far, everything we've done has been *out of the box* looking at different NEON sites, but without changing anything in the underlying model code.  You may want to do model experiments where you alter the vegetation growing at a site, modify some of the model parameters, modify namelist settings, change the input data, or even alter model code.  We'll get into how do make these changes later, but for now we'll get a test case set up.

Since we're already run an out of the box case for KONZ, we can create a paired experimental case at the same site.

<div class="alert alert-block alert-info" markdown='1'>
<b>RECOMMENDATION:</b> use a short, descriptive name for your experiment, it will help you down the road.
</div>


This example below just builds on what you've already been doing:

In [ ]:
# Change the 4-character NEON site below.
cd ~/CTSM/tools/site_and_regional
export new_case='CPER'    # the new site you want to run

# then run_neon
./run_neon.py --neon-sites $new_case  \
   --output-root ~/scratch/NEON_cases/ \
   --base-case ~/scratch/NEON_cases/$neon_site.$experiment.transient \
   --experiment $experiment \
   --setup-only \
   --overwrite \
   --prism

<div class="alert alert-block alert-warning" markdown='1'>

<b>WARNING:</b> Because we're also using the `--base-case` flag, we won't have to rebuild our experimental case.  This may not be advisable if you're modifying model code.

</div>

At this point your experimental case has been created.  
- What is the case name?
- Can you navigate to your case directory?
- Are there any differences between this experimental case and the base case you already ran?
- Can you find the datm input data for your case?
- Is the model going to use an initial conditions file?
- Where is the surface dataset that's being used?
- Can you find the parameter file for your case?

---

Now we'll point to the correct restart file and submit the case

In [ ]:
cd ~/scratch/NEON_cases/$new_case.$experiment.transient
echo "finidat='/scratch/data/NEONv2/PRISM.restart/$new_case.postad.clm2.r.0318-01-01-00000.nc'" >> user_nl_clm

#Then check it worked as intended
./preview_namelists

cat CaseDocs/lnd_in | grep finidat

In [ ]:
cat CaseDocs/lnd_in | grep finidat
./case.submit


<div class="alert alert-success">
<strong>Congratulations!</strong> 
    
You've created and run an experimental case that uses PRISM precipitation data instead of NEON precipitation
    .
</div>

